# 超参调节、批量标准化、编程框架

## 1. 超参调节

### 1.1 调参过程

调参的优先级：

1）学习速率 $\alpha$

2）如果使用动量梯度下降 $\beta$；隐藏神经元数量；微批大小。

3）神经网络的层数；学习速率衰减

4）如果使用Adam，几乎不太会去调 $\beta_1=0.9, \beta_2=0.999, \epsilon=10^{-8}$

在机器学习的早期阶段，通常会使用网格搜索。当超参较少的情况下，网格搜索是没问题的。对于神经网络这样超参众多的模型，更推荐随机挑选。因为通常很难预先知道哪一个超参会更重要，实际上超参发挥的作用，会有很大的差距，会重要的超参尝试更多的值搜索效果会更好。

![Try random values, don't use a grid](img/Try random values, don't use a grid.png)

另一个推荐的搜索策略，是采用由粗到细的搜索（coarse to fine），在表现较好的超参区域，放大，进一步搜索。

![Coarse to fine](img/Coarse to fine.png)